In [1]:
import json
import urllib.request
import csv

In [2]:
# steps:
# import JSON file
# parse the JSON file to get the info we want
# write those coordinates to a csv
# export the csv

In [3]:
#the first function here calls all the helpers underneath at different times. The 'names' variable holds all the 
# information to be printed. The rest of the 'export' function prints this to a csv with the field names on top, using the
# writer and dictwriter functions.

def export_coordinates(input_list, filename):
    names = getinfo(input_list)
    output_filename = filename+'.csv'
    with open(output_filename, mode='w', newline='') as csvfile:
        fieldnames = names[0].keys()
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fieldnames)
        dictwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for row in names:
            dictwriter.writerow(row)
            
#this function first creates an empty list, then fills it with dictionaries. The dictionaries will each represent one site
#from Pleiades. getting the info happens in the helpers underneath
            
def getinfo(input_list):
    names = []
    for ids in input_list :
        one_site = {}
        one_site['title'] = getname(ids)
        one_site['pleiadesID'] = ids
        one_site['longitude'] = getcords(ids)[0]
        one_site['latitude'] = getcords(ids)[1]
        names.append(one_site)
    return names

#the main helper. This returns the JSON file from Pleiades' database. If the URI is different from this, I don't know how to 
#get the information.
def getjson(uri) :
    uri = f'https://pleiades.stoa.org/places/{uri}/json'
    with urllib.request.urlopen(uri) as f:
        data = json.loads(f.read().decode())
    return data

#getname and getcords both do the same thing for different keys. Each check to see if there is a value associated with the 
#key we want to find (first romanized name, then representative point), and then return either that value or "na"
def getname(uri) : 
    if bool(getfirstname(getjson(uri).get('names')[0].get('romanized'))):
        return getfirstname(getjson(uri).get('names')[0].get('romanized'))
    else:
        return "NA"
    
def getcords(uri):
    if bool(getjson(uri).get('reprPoint')):
        return getjson(uri).get('reprPoint')
    else :
        return ['NA','NA']
    
#if there are multiple romanized names, this says just get the first    
def getfirstname(string):
    return string.partition(",")[0]

In [4]:
#a tester case
testerids = [579885, 589918, 570685, 541138, 727070]

In [5]:
export_coordinates(testerids, 'Athens, Lytos, Sparta, Thebes, Alexandria')